In [1]:
import pandas as pd
import numpy as np

In [2]:
ma_hyperparams_df = pd.read_csv("moving_averages_df.csv")

In [20]:
ma_hyperparams_df.head(10)

,grain,sma_best_param,wma_best_param,ewma_best_param
0,10_21,2.0,2.0,2.0
1,10_37,6.0,8.0,7.0
2,10_5,9.0,9.0,9.0
3,10_86,2.0,2.0,2.0
4,10_93,2.0,2.0,2.0
5,11_110,5.0,5.0,5.0
6,11_61,2.0,2.0,2.0
7,11_68,3.0,9.0,9.0
8,11_86,2.0,2.0,2.0
9,11_87,5.0,5.0,5.0


#### How to perform forecasting for a given date for a given grain
* check the grain and get the hyper_parameters corresponding to each algorithm.
* for each date corresponding to test data, get all the prior date entries(window-size) required for each algorithm.
* Create a window of units for those prior date entries, once that is done.

In [4]:
test_df = pd.read_csv("../csv_files/test.csv")

In [5]:
test_df['grain'] = test_df['store_nbr'].astype('str') + '_' + test_df['item_nbr'].astype('str')

In [6]:
test_df['date'] = pd.to_datetime(test_df['date'])

In [11]:
grain_15_45_test = test_df[test_df['grain'] == '15_45'].reset_index()

In [32]:
train_weather_df = pd.read_csv('train_weather_df.csv')
train_weather_df['date'] = pd.to_datetime(train_weather_df['date'])

In [38]:
import pickle

In [39]:
file = open('trained_regressors_dict','rb')
trained_regressors_dict = pickle.load(file)
file.close()

In [40]:
grain_15_45_regressor = trained_regressors_dict['15_45']

In [13]:
from datetime import timedelta

In [52]:
def get_sma(units):
    return np.mean(units)

In [91]:
def get_wma(units):
    window_size = len(units)
    win_index = 0
    window_avg = 0
    for unit in units:
        window_avg += (window_size-win_index)*unit
        win_index += 1
    window_avg /= (window_size*(window_size+1)/2)
    return window_avg

In [65]:
def get_ewma(units):
    window_size = len(units)
    alpha = 2/(window_size+1)
    win_index = 0
    ewa_num = 0
    ewa_denom = 0
    ewa_avg = 0
    for unit in units:
        ewa_num += (((1 - alpha)**win_index) * unit)
        ewa_denom += ((1 - alpha)**win_index)
        win_index += 1
    ewa_avg = ewa_num/ewa_denom
    return ewa_avg

In [85]:
def get_window_dates(date, window_size):
    return [date - timedelta(days = window) for window in range(1,window_size+1)]

In [66]:
global_units_index = dict()

In [69]:
def get_input_units(grain, date, window_size, algo):
    window_dates = get_window_dates(date, window_size)
    required_units = []
    history_count = 0
    predicted_count = 0
    
    for window_date in window_dates:
        if window_date in train_weather_df.date.values:
            unit = train_weather_df[train_weather_df['date'] == window_date]['units'].values[0]
            if window_date not in global_units_index.keys():
                global_units_index[(grain,window_date,algo)] = unit
            history_count += 1
            required_units.append(unit)    
        else:
            unit = global_units_index[(grain,window_date,algo)]
            predicted_count +=1
            required_units.append(unit)
    return required_units

In [93]:
predictions = []
grain = '15_45'

sma_window = int(ma_hyperparams_df[ma_hyperparams_df['grain'] == grain]['sma_best_param'].values[0])
wma_window = int(ma_hyperparams_df[ma_hyperparams_df['grain'] == grain]['wma_best_param'].values[0])
ewma_window = int(ma_hyperparams_df[ma_hyperparams_df['grain'] == grain]['ewma_best_param'].values[0])

date = grain_15_45_test.date[0]
sma_units = get_input_units(grain, date, sma_window)
pred = get_sma(required_units)
print("SMA prediction : ",pred)
global_units_index[(grain,date,'sma')] = pred

wma_units = get_input_units(grain, date, wma_window)
pred = get_wma(required_units)
print("WMA prediction : ",pred)
global_units_index[(grain,date,'wma')] = pred

ewma_units = get_input_units(grain, date, ewma_window)
pred = get_ewma(required_units)
print("EWMA prediction : ",pred)
global_units_index[(grain,date,'ewma')] = pred

SMA prediction :  73.96875
WMA prediction :  74.0
EWMA prediction :  74.01562499999999
